# Student Loan Risk with Deep Learning

In [2]:
!pip install tensorflow

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   - -------------------------------------- 10.7/390.3 MB 67.2 MB/s eta 0:00:06
   -- ------------------------------------- 27.3/390.3 MB 75.2 MB/s eta 0:00:05
   ---- ----------------------------------- 46.9/390.3 MB 83.0 MB/s eta 0:00:05
   ------ --------------------------------- 63.2/390.3 MB 85.7 MB/s eta 0:00:04
   ------- -------------------------------- 77.9/390.3 MB 80.1 MB/s eta 0:00:04
   --------- ------------------------------ 96.2/390.3 MB 81.9 MB/s eta 0:00:04
   ----------- --------------------------- 113.8/390.3 MB 82.6 MB/s eta 0:00:04
   ------------- ------------------------- 132.4/390.3 MB 82.9 MB/s eta 0:00:04
   --------------- ----------------------- 150.7/390.3 MB 83.7 MB/s eta 0:00:03
   ---------------- ---------------------- 169.9/390.3 MB 84.8 MB/s eta 0:00:03
   ------------------- ------------------- 191.6/390

In [15]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [49]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


In [51]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [53]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [58]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"] 

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [60]:
# Define features set X by selecting all columns but credit_ranking
features = X = loans_df.drop(columns=["credit_ranking"])


# Review the features DataFrame
features

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [63]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [45]:
# Create a StandardScaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)



---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [84]:
# Define the the number of inputs (features) to the model
def create_model(hp):
    nn_model = tf.keras.models.Sequential("features")

# Review the number of features
features


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [90]:
# Define the number of hidden nodes for the first hidden layer
models.add(Dense(64, input_dim=10, activation='relu'))

# Define the number of hidden nodes for the second hidden layer
models.add(Dense(32, activation='relu'))

# Define the number of neurons in the output layer
models.add(Dense(1, activation='sigmoid'))

NameError: name 'models' is not defined

In [11]:
# Create the Sequential model instance


# Add the first hidden layer


# Add the second hidden layer


# Add the output layer to the model specifying the number of output neurons and activation function


In [12]:
# Display the Sequential model summary


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model


In [14]:
# Fit the model using 50 epochs and the training data


Epoch 1/50
38/38 [==============================] - 3s 5ms/step - loss: 0.6736 - accuracy: 0.5696
Epoch 2/50
38/38 [==============================] - 0s 5ms/step - loss: 0.6642 - accuracy: 0.6397
Epoch 3/50
38/38 [==============================] - 0s 6ms/step - loss: 0.6551 - accuracy: 0.6814
Epoch 4/50
38/38 [==============================] - 0s 6ms/step - loss: 0.6467 - accuracy: 0.6939
Epoch 5/50
38/38 [==============================] - 0s 5ms/step - loss: 0.6390 - accuracy: 0.6989
Epoch 6/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6319 - accuracy: 0.7098
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6255 - accuracy: 0.7173
Epoch 8/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6193 - accuracy: 0.7164
Epoch 9/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6139 - accuracy: 0.7231
Epoch 10/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6084 - accuracy: 0.7281
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [15]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data


# Display the model loss and accuracy results


13/13 - 0s - loss: 0.5049 - accuracy: 0.7350 - 175ms/epoch - 13ms/step
Loss: 0.5049149394035339, Accuracy: 0.7350000143051147


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [16]:
# Set the model's file path


# Export your model to a keras file


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [17]:
# Set the model's file path


# Load the model to a new object


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [18]:
# Make predictions with the test data


# Display a sample of the predictions


13/13 - 0s - 126ms/epoch - 10ms/step


array([[0.63573396],
       [0.41001907],
       [0.88199055],
       [0.61770827],
       [0.9708398 ]], dtype=float32)

In [19]:
# Save the predictions to a DataFrame and round the predictions to binary results


,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [20]:
# Print the classification report with the y test data and predictions


              precision    recall  f1-score   support

           0       0.70      0.76      0.73       188
           1       0.77      0.72      0.74       212

    accuracy                           0.73       400
   macro avg       0.74      0.74      0.73       400
weighted avg       0.74      0.73      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

